# SageMaker JumpStart - deploy automatic speech recognition model

This notebook demonstrates how to use the SageMaker Python SDK to deploy a JumpStart automatic speech recognition model.

In [3]:
import sagemaker
import time
import boto3
import json
from sagemaker.utils import name_from_base

sess = sagemaker.Session()
bucket = sess.default_bucket() # Set a default S3 bucket
prefix = 'whisper-sm-js'

# S3 client
s3 = boto3.client('s3')

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [4]:
import json

import boto3
from sagemaker.jumpstart.model import JumpStartModel
from sagemaker.jumpstart.utils import get_jumpstart_content_bucket

Select your desired model ID. You can search for available models in the [Built-in Algorithms with pre-trained Model Table](https://sagemaker.readthedocs.io/en/stable/doc_utils/pretrainedmodels.html).

In [5]:
model_id = "huggingface-asr-whisper-small"

#"huggingface-asr-whisper-large-v3"

## > Predict Async with SageMaker SDK

In [6]:
from sagemaker.async_inference import AsyncInferenceConfig

# Create an AsyncInferenceConfig object
async_config = AsyncInferenceConfig(
    output_path=f"s3://{bucket}/{prefix}/output", 
    max_concurrent_invocations_per_instance = 4,
    # notification_config = {
            #   "SuccessTopic": "arn:aws:sns:us-east-2:123456789012:MyTopic",
            #   "ErrorTopic": "arn:aws:sns:us-east-2:123456789012:MyTopic",
    # }, #  Notification configuration 
)

In [7]:
model = JumpStartModel(model_id=model_id)

Using model 'huggingface-asr-whisper-small' with wildcard version identifier '*'. You can pin to version '3.0.0' for more stable results. Note that models may have different input/output signatures after a major version upgrade.


In [15]:
# Deploy the model for async inference
endpoint_name = name_from_base(f"{prefix}-async-endpoint")

async_predictor = model.deploy(
    async_inference_config=async_config,
    instance_type="ml.g5.2xlarge",
    initial_instance_count=1,
    endpoint_name=endpoint_name,
)

Using already existing model: hf-asr-whisper-small-2024-06-13-23-28-54-108


----------!

## > Download sample audio file

In [16]:
def download_and_load_audio_file(file_name):
    s3_bucket = get_jumpstart_content_bucket()
    s3_prefix = "training-datasets/asr_notebook_data"
    s3_client = boto3.client("s3")
    s3_client.download_file(s3_bucket, f"{s3_prefix}/{file_name}", file_name)
    with open(file_name, "rb") as file:
        input_audio = file.read()
    return input_audio

audio_file = "sample1.wav"
input_audio = download_and_load_audio_file(audio_file)

## Invoke

In [18]:
response = async_predictor.predict_async(
    data=input_audio,
    initial_args={'ContentType': 'audio/wav'})
print(response.output_path)

s3://sagemaker-us-west-2-376678947624/whisper-sm-js/output/15e4b93e-9437-496a-a9ee-6325ad00954e.out


## > Invoke in batch

In [19]:
## upload the sample file
s3_key = f"{prefix}/input/{audio_file}"
s3.upload_file(audio_file, bucket, s3_key)

In [20]:
num_duplicates = 10

# Create duplicate copies
for i in range(num_duplicates):
    duplicate_key = f"{s3_key.rsplit('.', 1)[0]}_{i+1}.{s3_key.rsplit('.', 1)[1]}"
    copy_source = {
        'Bucket': bucket,
        'Key': s3_key
    }
    s3.copy_object(Bucket=bucket, Key=duplicate_key, CopySource=copy_source)
    print(f"Created duplicate copy s3://{bucket}/{duplicate_key}")

Created duplicate copy s3://sagemaker-us-west-2-376678947624/whisper-sm-js/input/sample1_1.wav
Created duplicate copy s3://sagemaker-us-west-2-376678947624/whisper-sm-js/input/sample1_2.wav
Created duplicate copy s3://sagemaker-us-west-2-376678947624/whisper-sm-js/input/sample1_3.wav
Created duplicate copy s3://sagemaker-us-west-2-376678947624/whisper-sm-js/input/sample1_4.wav
Created duplicate copy s3://sagemaker-us-west-2-376678947624/whisper-sm-js/input/sample1_5.wav
Created duplicate copy s3://sagemaker-us-west-2-376678947624/whisper-sm-js/input/sample1_6.wav
Created duplicate copy s3://sagemaker-us-west-2-376678947624/whisper-sm-js/input/sample1_7.wav
Created duplicate copy s3://sagemaker-us-west-2-376678947624/whisper-sm-js/input/sample1_8.wav
Created duplicate copy s3://sagemaker-us-west-2-376678947624/whisper-sm-js/input/sample1_9.wav
Created duplicate copy s3://sagemaker-us-west-2-376678947624/whisper-sm-js/input/sample1_10.wav


In [21]:
response = s3.list_objects_v2(Bucket=bucket, Prefix=f"{prefix}/input/")

for obj in response.get('Contents', []):
    key = obj['Key']
    input_path = f"s3://{bucket}/{key}"

    
    response = async_predictor.predict_async(
        input_path=input_path,
        initial_args={'ContentType': 'audio/wav'})

    print(response.output_path)

s3://sagemaker-us-west-2-376678947624/whisper-sm-js/output/6861581c-6287-4fba-89ce-e16d4112b2a3.out
s3://sagemaker-us-west-2-376678947624/whisper-sm-js/output/216387e5-58d7-4488-b927-1d9cc72f08d1.out
s3://sagemaker-us-west-2-376678947624/whisper-sm-js/output/0262652a-2626-4ff8-9f65-65be65a0ba82.out
s3://sagemaker-us-west-2-376678947624/whisper-sm-js/output/700c5910-e697-4625-beb3-76fcab86c1e4.out
s3://sagemaker-us-west-2-376678947624/whisper-sm-js/output/319b7a9a-183c-4526-87ee-4d2da1e08dba.out
s3://sagemaker-us-west-2-376678947624/whisper-sm-js/output/597eeb17-caa1-460b-aa67-e9a06c1ed1a0.out
s3://sagemaker-us-west-2-376678947624/whisper-sm-js/output/06565fc8-66d4-4f9c-85d5-baefd790a095.out
s3://sagemaker-us-west-2-376678947624/whisper-sm-js/output/bdfb606d-ecda-4c2e-9588-48c4daee833c.out
s3://sagemaker-us-west-2-376678947624/whisper-sm-js/output/b57a3152-dc25-46e2-87fd-34b078701073.out
s3://sagemaker-us-west-2-376678947624/whisper-sm-js/output/409e54bc-79bb-479f-9ad0-e2a23340b445.out


## > Clean up

In [22]:
async_predictor.delete_endpoint()